In [1]:
import tensorflow as tf
import numpy as np

# 设置numpy浮点数的长度
np.set_printoptions(precision=3, suppress=True)

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### 二元交叉熵损失

+ y_pred是样本属于类别1的分数
+ from_logits=True表示分数需要经过sigmoid函数概率化
+ reduction参数设置为None，表示返回所有样本的损失

In [3]:
bce_auto = tf.keras.losses.BinaryCrossentropy(from_logits=True)

bce_none = tf.keras.losses.BinaryCrossentropy(from_logits=True,
                reduction=tf.keras.losses.Reduction.NONE)

bce_sum = tf.keras.losses.BinaryCrossentropy(from_logits=True,
                reduction=tf.keras.losses.Reduction.SUM)

+ 假设现在有4个样本，类别分别是0, 0, 1, 1
+ BinaryCrossentropy()是以batch为计算单位，如果是一维数组，相当于只有一个batch

In [4]:
# 这样的二维数据相当于4个batch
y_true_1 = [[0], [1], [0], [0]]
y_pred_1 = [[-18.6], [0.51], [2.94], [-12.8]]

In [5]:
# 这样的一维数据其实就是相当于一个batch
y_true_2 = [0, 1, 0, 0]
y_pred_2 = [-18.6, 0.51, 2.94, -12.8]

In [6]:
# 2个batch
y_true_3 = [[0, 1], [0, 0]]
y_pred_3 = [[-18.6, 0.51], [2.94, -12.8]]

In [7]:
# 各个样本的损失值，通过API的计算结果如下
bce_auto(y_true_1, y_pred_1).numpy()

0.865458

In [8]:
bce_none(y_true_1, y_pred_1).numpy()

array([0.   , 0.47 , 2.992, 0.   ], dtype=float32)

In [9]:
bce_sum(y_true_1, y_pred_1).numpy()

3.461832

对第二批数据进行计算，三种reduction结果都一样，因为第二批数据只有一个batch，而bce函数实际上只会计算每个batch的损失值(注意这里每个batch的损失值是指这个batch中所有样本损失值的平均值)，然后再根据reduction参数，是返回每个batch的损失值还是求和，或者计算全体batch的平均值(reduction=auto)

In [10]:
bce_auto(y_true_2, y_pred_2).numpy()

0.865458

In [11]:
bce_none(y_true_2, y_pred_2).numpy()

0.865458

In [12]:
bce_sum(y_true_2, y_pred_2).numpy()

0.865458

第三批数据的计算结果，关于reduction参数只需要记住一点，就是bce函数是在batch这个维度上对损失值进行处理的，而不是对每个样本进行单独的计算，而在batch这个维度上，batch的损失值是该batch中所有样本损失的平均值

In [13]:
# 所有batch损失值的平均值
bce_auto(y_true_3, y_pred_3).numpy()

0.865458

In [14]:
# 每个batch的损失值
bce_none(y_true_3, y_pred_3).numpy()

array([0.235, 1.496], dtype=float32)

In [15]:
# 所有batch损失值的和
bce_sum(y_true_3, y_pred_3).numpy()

1.730916

### 现在手动模拟一遍，使用sigmoid函数将预测值概率化

In [16]:
y_true_2 = [0, 1, 0, 0]
y_pred_2 = [-18.6, 0.51, 2.94, -12.8]

In [17]:
# t是样本属于1类别的概率
t = tf.keras.activations.sigmoid(tf.constant(y_pred_2))
t.numpy()

array([0.   , 0.625, 0.95 , 0.   ], dtype=float32)

In [18]:
y_true_2 = [0, 1, 0, 0]

得到概率值后，下面计算交叉熵损失，交叉熵损失的公式是：$-p(x)logq(x)$, 二分类问题中p(x)=1, 而q(x)是样本正确类别的概率，例如样本x_1是1类别，那么q(x_1)就是其属于1类别的概率，而与其属于0类别的概率无关，即只关注样本对应的正确类别的概率(这一点在softmax的多分类交叉熵中是一样的)

由于二分类问题没有使用独热编码，因此损失函数一般写成$: -[p(x)logq(x) + (1-p(x))log(1-q(x))]$

In [19]:
def crossEntropy(y_true, y_pred):
    return -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

# 根据样本的正确类别计算每个样本的交叉熵损失，可以看到与API计算的结果一致
crossEntropy(np.array(y_true_2), t)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.   ,  0.47 ,  2.992,  0.   ], dtype=float32)>